# panogeo demo

End-to-end demo of detection and geolocation from panoramic imagery.

Requirements:
- Install environment: `conda env create -f environment.yml && conda activate panogeo`
- Install package: `pip install -e .`
- Prepare a folder of panoramas (equirectangular 2:1)



In [1]:
# Imports & paths
from pathlib import Path
from IPython.display import display

import pandas as pd

from panogeo.cli import main as panogeo_cli

# Set your paths
IMAGES_DIR = "./data/images"           # input panoramas
SHIFT_DIR  = "./data/images_shift"     # shifted output
OUTPUT_DIR = "./output"                # outputs
CALIB_CSV  = f"{OUTPUT_DIR}/calib_points.csv"  # provide or capture via your own tool

Path(SHIFT_DIR).mkdir(parents=True, exist_ok=True)
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

print("Setup complete.")


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\kunih\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Setup complete.


In [3]:
# 1) Optional: yaw-shift panoramas so forward faces North (or desired yaw)
panogeo_cli(["shift-pano", "--in-dir", IMAGES_DIR, "--out-dir", SHIFT_DIR, "--degrees", "200"])


Saved 36 image(s) to ./data/images_shift


In [4]:
# 2) Detect people in panoramas with tiling
panogeo_cli([
    "detect",
    "--images-dir", SHIFT_DIR,
    "--output-dir", OUTPUT_DIR,
    "--model", "yolov8s.pt",
    "--conf", "0.20",
    "--iou", "0.50",
])

# Inspect aggregate detections
agg_csv = f"{OUTPUT_DIR}/detections/detections_all.csv"
df_det = pd.read_csv(agg_csv)
display(df_det.head())


Detections saved: ./output\detections\detections_all.csv


,image,input_path,W,H,bbox_x1,bbox_y1,bbox_x2,bbox_y2,u_px,v_px,conf
0,IMG_20250906_110448_00_008,./data/images_shift\IMG_20250906_110448_00_008...,11904,5952,5118.839844,3133.550293,5230.365234,3321.864258,5174.602539,3321.864258,0.886771
1,IMG_20250906_110448_00_008,./data/images_shift\IMG_20250906_110448_00_008...,11904,5952,2915.220215,3134.776367,3010.488525,3282.270996,2962.854370,3282.270996,0.828333
2,IMG_20250906_110448_00_008,./data/images_shift\IMG_20250906_110448_00_008...,11904,5952,5252.989258,3024.637451,5291.880859,3092.975342,5272.435059,3092.975342,0.819886
3,IMG_20250906_110448_00_008,./data/images_shift\IMG_20250906_110448_00_008...,11904,5952,3735.650391,3098.067627,3824.960938,3307.867920,3780.305664,3307.867920,0.790191
4,IMG_20250906_110448_00_008,./data/images_shift\IMG_20250906_110448_00_008...,11904,5952,2481.886230,4159.875977,2819.313477,4319.469238,2650.599854,4319.469238,0.786362


In [ ]:
# 3) Calibrate camera rotation from pixel↔geo pairs
# Provide a CSV with columns: image,u_px,v_px,lon,lat[,alt_m][,W,H]
# Set your known camera location and altitude
CAM_LAT       = 35.169237383755025
CAM_LON       = 136.90874779113642
CAMERA_ALT_M  = 2.2       # camera altitude above local ground mean sea level (approx). Used only to set camera Z in ENU.
GROUND_ALT_M  = 0.0       # assumed ground altitude (MSL) when calib alt is absent

panogeo_cli([
    "calibrate",
    "--calib-csv", CALIB_CSV,
    "--cam-lat", str(CAM_LAT),
    "--cam-lon", str(CAM_LON),
    "--camera-alt-m", str(CAMERA_ALT_M),
    "--ground-alt-m", str(GROUND_ALT_M),
    "--output-dir", OUTPUT_DIR,
])


In [ ]:
# 4) Geolocate detections to ENU and WGS84
calib_npz = f"{OUTPUT_DIR}/calibration_cam2enu.npz"
panogeo_cli([
    "geolocate",
    "--detections-csv", agg_csv,
    "--calibration", calib_npz,
    "--output-dir", OUTPUT_DIR,
])

geo_csv = f"{OUTPUT_DIR}/all_people_geo_calibrated.csv"
df_geo = pd.read_csv(geo_csv)
print(f"Rows: {len(df_geo)}")
display(df_geo.head())


In [ ]:
# 5) Folium heatmap (requires folium)
try:
    import folium
    from folium.plugins import HeatMap
    m = folium.Map(location=[float(df_geo["lat"].mean()), float(df_geo["lon"].mean())], zoom_start=19, tiles=None)
    folium.TileLayer(
        tiles="https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png",
        attr='© OpenStreetMap contributors © CARTO',
        name="CartoDB Positron",
    ).add_to(m)
    heat_data = [[float(r.lat), float(r.lon), float(getattr(r, "conf", 1.0))] for r in df_geo.itertuples()]
    HeatMap(heat_data, radius=12, blur=18, min_opacity=0.25, max_zoom=21).add_to(m)
    m
except Exception as e:
    print("Folium not available:", e)
